<a href="https://colab.research.google.com/github/xlnt415/xlnt_portfolio.github.io/blob/main/code%20/side_project/%EC%83%88%EB%A1%9C%EC%9A%B4%EA%B0%80%EA%B5%AC%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

In [ ]:
# 라이브러리 불러오기
import json
import operator
import pandas as pd
from datetime import datetime, timedelta
import re
from collections import Counter
import seaborn as sns
from matplotlib.dates import DateFormatter
import numpy as np

In [ ]:
# matplotlib 한글화
!pip install koreanize-matplotlib

In [ ]:
# 한글 폰트 설정
import matplotlib.pyplot as plt
import koreanize_matplotlib
import matplotlib.patches as mpatches

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

os.chdir('/content/drive/MyDrive/취업준비/오늘의집/원본데이터')
# os.chdir(r'C:\Users\user\Google Drive 스트리밍\내 드라이브\취업준비\오늘의집\원본데이터')

# 1. 인테리어 산업 현황

##  1) 인테리어 산업 매출

In [ ]:
interior_sales = pd.read_excel('인테리어업체매출액.xlsx')

In [ ]:
interior_sales

In [ ]:
interior_sales.plot(x = '시점', y = '매출액평균') # 출처 : 국가통계 포털

# 2. 가구형태 분석

## 1) 가구원수 변화

In [ ]:
type_furniture = pd.read_excel('가구원수별.xlsx')

In [ ]:
type_furniture.head()

In [ ]:
grouped_data = type_furniture.melt(id_vars = ['시점', '행정구역별(읍면동)'], var_name = '가구원수', value_name = '가구수')
grouped_data.head()

In [ ]:
grouped_data_nation = grouped_data[grouped_data['행정구역별(읍면동)'] == '전국']
grouped_data_nation.head()

In [ ]:
grouped_data_nation.pivot_table(index = '시점', columns='가구원수', values='가구수', aggfunc = np.mean)

In [ ]:
# 2017년 대비 증감률
grouped_data_nation_inde = pd.read_excel('가구형태/가구원수별_2017_대비_증감률.xlsx')
grouped_data_nation_inde

- 1인가구와 2인가구의 상승

## 2) 가구형태 변화

In [ ]:
Age_Household_Type = pd.read_excel('가구형태/가구주의_성__연령_및_세대구성별_가구_일반가구.xlsx')

In [ ]:
# 친인척, 1인가구, 비친인척 나누기
Age_Household_Type.info()

In [ ]:
Age_Household_Type.drop(columns='일반가구', axis = 1, inplace=True)

In [ ]:
# Select columns to concatenate horizontally
columns_to_concat = Age_Household_Type.iloc[:, [2, 9, 19, 24]]

# Sum along the columns (axis=1 for horizontal sum)
result = columns_to_concat.sum(axis=1)

# Assign the result to a new column '가족' in Age_Household_Type DataFrame
Age_Household_Type['가족'] = result

In [ ]:
Age_Household_Type_melt = Age_Household_Type.iloc[:, [0, 1, 25, 26, 27]].melt(id_vars = ['시점', '가구주의 연령별'], value_vars=['비친족가구', '가족'], var_name='가구형태', value_name='가구수')
Age_Household_Type_melt

In [ ]:
Age_Household_Type_melt_pivot = Age_Household_Type_melt.pivot_table(index = '시점', columns = '가구형태', values='가구수', aggfunc = np.sum)
Age_Household_Type_melt_pivot

In [ ]:
Age_Household_Type_melt_pivot['가족_비율'] = ((Age_Household_Type_melt_pivot['가족'] - Age_Household_Type_melt_pivot['가족'].iloc[0]) / Age_Household_Type_melt_pivot['가족'].iloc[0]) * 100
Age_Household_Type_melt_pivot['비친족가구_비율'] = ((Age_Household_Type_melt_pivot['비친족가구'] - Age_Household_Type_melt_pivot['비친족가구'].iloc[0]) / Age_Household_Type_melt_pivot['비친족가구'].iloc[0]) * 100

In [ ]:
# 2018년 대비 증감률
Age_Household_Type_melt_pivot.iloc[1:, 2:]

In [ ]:
# 2017년 대비 증감률
Age_Household_Type_inde = pd.read_excel('가구형태/가구주의_성__연령_및_세대구성별_가구_일반가구_2017_대비_증감률.xlsx')

In [ ]:
Age_Household_Type_inde.info()

In [ ]:
Age_Household_Type_inde.drop(columns='일반가구', axis = 1, inplace=True)

In [ ]:
# Select columns to concatenate horizontally
columns_to_concat = Age_Household_Type_inde.iloc[:, [2, 9, 19, 24]]

# Sum along the columns (axis=1 for horizontal sum)
result = columns_to_concat.sum(axis=1)

# Assign the result to a new column '가족' in Age_Household_Type DataFrame
Age_Household_Type_inde['가족'] = result

In [ ]:
Age_Household_Type_inde_melt = Age_Household_Type_inde.iloc[:, [0, 1, 25, 26, 27]].melt(id_vars = ['시점', '가구주의 연령별'], value_vars=['비친족가구', '가족'], var_name='가구형태', value_name='증감률')
Age_Household_Type_inde_melt

In [ ]:
Age_Household_Type_inde_melt.pivot_table(index = '시점', columns = '가구형태', values='증감률', aggfunc = np.mean)

- 위 결과 비친족, 2인 가구가 상승하고 있음을 알 수 있음.

## 3) 비친족가구 분석

In [ ]:
# 비친족가구만 분리
Age_Household_Type_nof = Age_Household_Type_melt[Age_Household_Type_inde_melt.가구형태 == '비친족가구']
Age_Household_Type_nof

In [ ]:
Age_Household_Type_nof.pivot_table(index = '시점', columns = '가구주의 연령별', values='가구수', aggfunc = np.mean)

In [ ]:
Age_Household_Type_inde_nof = Age_Household_Type_inde_melt[Age_Household_Type_inde_melt.가구형태 == '비친족가구']
Age_Household_Type_inde_nof

In [ ]:
Age_Household_Type_inde_nof.pivot_table(index = '시점', columns = '가구주의 연령별', values='증감률', aggfunc = np.mean)

- 25세에서 39세 사이에서 높은 증가율을 보여줌

## 4) 2030이 가장 많이 소유 하고 있는 면적

In [ ]:
Age_House_Square_Footage = pd.read_excel('주택면적_연령대별_주택소유자수.xlsx')

In [ ]:
# category_mapping = {
#     "시점" : "시점",
#     "연령대별" : "연령대별",
#     "12평 이하" : "12평 이하",
#     "12평 초과 18평 이하" : "12평 초과 18평 이하",
#     "18평 초과 30평 이하": "18평 초과",
#     "30평 초과 50평 이하": "18평 초과",
#     "50평 초과": "18평 초과"
# }

# Age_House_Square_Footage = Age_House_Square_Footage.rename(columns=category_mapping)

In [ ]:
#  '연령대별' 칼럼에서 '29세이하', '30~39세' 값의 행 추출
Age_House_Square_Footage_2030 = Age_House_Square_Footage[Age_House_Square_Footage['연령대별'].isin(['29세이하', '30~39세'])]

In [ ]:
Age_House_Square_Footage_2030.info()

In [ ]:
# Age_House_Square_Footage['test'] = np.sum(Age_House_Square_Footage.iloc[:,4:], axis = 1)

In [ ]:
Age_House_Square_Footage_2030 = Age_House_Square_Footage_2030[Age_House_Square_Footage_2030.시점 > 2017]

In [ ]:
Age_House_Square_Footage_2030.sort_values(by = '연령대별')

20대는 12 ~ 30평, 30대는 18 ~ 30평 사이의 집을 선호함

- 주택종류 : 아파트
- 면적 : 18평 초과
- 아파트 연식 : 2년~4년
- 타겟층 : 4060

2월과 3월에 전입 인구가 가장 많음

## 5) 주거선택 이유

In [ ]:
Reasons_residence = pd.read_excel('현재_거주_중인_주택_선택_이유.xlsx')

In [ ]:
Reasons_residence.iloc[1:, :]

2020년 조사 결과 이들은 주택위치, 주거비용 순으로 주택을 선택함  -> 인테리어 관심도가 낮음(?)

## 6) 건축 연도 및 **평수**

In [ ]:
year_construction = pd.read_excel('노후건물/주택의_종류__연면적_및_건축연도별_주택.xlsx')

In [ ]:
year_construction.연면적별.unique()

In [ ]:
category_mapping = {
    "합계": "합계",
    "20㎡ 이하": "12평 이하",
    "20㎡ ~ 40㎡": "12평 이하",
    "40㎡ ~ 60㎡": "12평 초과 18평 이하",
    "60㎡ ~ 85㎡": "18평 초과",
    "85㎡ ~ 100㎡": "18평 초과",
    "100㎡ ~ 130㎡": "18평 초과",
    "130㎡ ~ 165㎡": "18평 초과",
    "165㎡ ~ 230㎡": "18평 초과",
    "230㎡ 초과": "18평 초과"
}

year_construction['연면적별'] = year_construction['연면적별'].map(category_mapping)

In [ ]:
# Define the mapping for decades
decade_mapping = {
    '주택_계': None,
    2010.0: '2010년대', 2011.0: '2010년대', 2012.0: '2010년대', 2013.0: '2010년대', 2014.0: '2010년대',
    2015.0: '2010년대', 2016.0: '2010년대', 2017.0: '2010년대', 2018.0: '2010년대', 2019.0: '2010년대',
    2020.0: '2020년대', 2021.0: '2020년대', 2022.0: '2020년대',
    ' 2005~2009': '2000년대', ' 2000~2004': '2000년대',
    ' 1990~1999': '1990년대',
    ' 1980~1989': '1980년대',
    ' 1979 이전': '1979년 이전'
}
year_construction_2 = year_construction.copy()
# Apply the mapping to the '건축연도' column
year_construction_2['건축연도'] = year_construction['건축연도'].map(decade_mapping)

# Display the modified data
year_construction_2.head()

In [ ]:
year_construction_22 = year_construction_2[year_construction['시점'] == 2022]
year_construction_22 = year_construction_22[year_construction_22.연면적별 == '12평 초과 18평 이하'].drop(717, axis = 0)
year_construction_22['총주택수'] = year_construction_22['단독주택'] + year_construction_22['아파트'] + year_construction_22['연립주택'] + year_construction_22['다세대주택']

In [ ]:
year_construction_22_gb = pd.DataFrame(year_construction_22.groupby('건축연도')['총주택수'].sum())
year_construction_22_gb.sort_values(by = '총주택수', ascending=False)

주거 선택 이유를 살펴봤을 때, 2030세대는 1990년대 낡은 건물을 선택할 확률이 높음

In [ ]:
# # Filter the data for '12평 이하' apartments and exclude rows where '건축연도' is None
# filtered_small_apartments = year_construction_2[(year_construction_2['연면적별'] == '12평 이하') & (year_construction_2['건축연도'] != None)]

# # Group the filtered data by '시점' and '건축연도', then sum up the '아파트' column
# grouped_small_apartments = filtered_small_apartments.groupby(['시점', '건축연도'])['아파트'].sum().unstack(fill_value=0)

# # Display the grouped data
# grouped_small_apartments.head()

In [ ]:
# # Calculate the year-over-year growth rate
# grouped_small_apartments_rate = grouped_small_apartments.pct_change() * 100

# # Display the result
# grouped_small_apartments_rate.head()
# # 12평 이하 아파트의 수요가 많이 늘어났음

# 타겟분석

## 1) 타겟 재산과 부채

In [ ]:
finance = pd.read_excel('연령별_자산_부채_상환액.xlsx')

In [ ]:
finance_2030 = finance[finance.분류.isin(['자산 (만원)', '부채 (만원)'])].iloc[:, :4]
finance_2030

In [ ]:
assets = finance_2030[finance_2030['분류'].str.contains('자산')].reset_index()
debts = finance_2030[finance_2030['분류'].str.contains('부채')].reset_index()

In [ ]:
assets = assets.fillna(0)
debts = debts.fillna(0)

# Calculate the debt-to-assets ratio
debts_per = debts.iloc[:, 3:] / assets.iloc[:, 3:] * 100

In [ ]:
debts_per.round(1)

자산 대비 부채 비율이 높아지면서 주거 비용을 줄이고자 새로운 가구 형태를 선택했다고 봄

## 2) 소비 분석

In [ ]:
Consumption_expenditure_amount = pd.read_excel('타겟분석/가구특성별_소비지출구간별_가구분포.xlsx', engine='openpyxl')

In [ ]:
Consumption_expenditure_amount.가구특성별.unique()

In [ ]:
Consumption_expenditure_amount[Consumption_expenditure_amount.가구특성별.str.contains('29')]

In [ ]:
Consumption_expenditure_amount[Consumption_expenditure_amount.가구특성별.str.contains('39')]

In [ ]:
Consumption_expenditure_amount[Consumption_expenditure_amount.가구특성별.str.contains('2인')]

In [ ]:
Age_specific_per_household_average_household_balance = pd.read_excel('타겟분석/가구주_연령별_가구당_월평균_가계수지_전국_2인이상.xlsx', engine='openpyxl')

In [ ]:
Age_specific_per_household_average_household_balance_39 = Age_specific_per_household_average_household_balance[Age_specific_per_household_average_household_balance.가구주연령별.str.contains('39세')]
Age_specific_per_household_average_household_balance_39.head()

In [ ]:
Age_specific_per_household_average_household_balance = pd.read_excel('타겟분석/가구주_연령별_가구당_월평균_가계수지_전국_2인이상.xlsx', engine='openpyxl')
Age_specific_per_household_average_household_balance_39 = Age_specific_per_household_average_household_balance[Age_specific_per_household_average_household_balance.가구주연령별.str.contains('39세')]
Age_specific_per_household_average_household_balance_cos = Age_specific_per_household_average_household_balance_39[Age_specific_per_household_average_household_balance_39.가계수지항목별 == '소비지출']
Age_specific_per_household_average_household_balance_39_top10 = Age_specific_per_household_average_household_balance_39[
    Age_specific_per_household_average_household_balance_39['가계수지항목별'].str.contains('0') |
    Age_specific_per_household_average_household_balance_39['가계수지항목별'].str.contains('1')
]
Age_specific_per_household_average_household_balance_39_top10_pivot = Age_specific_per_household_average_household_balance_39_top10.pivot_table(index = '시점', columns = '가계수지항목별', values='전체가구', aggfunc = np.sum)


# 품목별 소비자 물가지수
Consumer_Price_Index= pd.read_excel('타겟분석/지출목적별_소비자물가지수.xlsx', engine='openpyxl')
Consumer_Price_Index = Consumer_Price_Index.iloc[1:, :]
Consumer_Price_Index.set_index('시점', inplace=True)
columns_name = Age_specific_per_household_average_household_balance_39_top10_pivot.columns
columns_name = list(columns_name)
Consumer_Price_Index.columns = columns_name


In [ ]:
Age_specific_per_household_average_household_balance_cos = Age_specific_per_household_average_household_balance_39[Age_specific_per_household_average_household_balance_39.가계수지항목별 == '소비지출']
Age_specific_per_household_average_household_balance_cos

In [ ]:
Age_specific_per_household_average_household_balance_39_top10 = Age_specific_per_household_average_household_balance_39[
    Age_specific_per_household_average_household_balance_39['가계수지항목별'].str.contains('0') |
    Age_specific_per_household_average_household_balance_39['가계수지항목별'].str.contains('1')
]

In [ ]:
Age_specific_per_household_average_household_balance_39_top10_pivot = Age_specific_per_household_average_household_balance_39_top10.pivot_table(index = '시점', columns = '가계수지항목별', values='전체가구', aggfunc = np.sum)
Age_specific_per_household_average_household_balance_39_top10_pivot

In [ ]:
# 품목별 소비자 물가지수
Consumer_Price_Index= pd.read_excel('타겟분석/지출목적별_소비자물가지수.xlsx', engine='openpyxl')

In [ ]:
Consumer_Price_Index = Consumer_Price_Index.iloc[1:, :]
Consumer_Price_Index.set_index('시점', inplace=True)

In [ ]:
columns_name = Age_specific_per_household_average_household_balance_39_top10_pivot.columns
columns_name = list(columns_name)
Consumer_Price_Index.columns = columns_name

In [ ]:
type(columns_name)

In [ ]:
Consumer_Price_Index

In [ ]:
# 물가 조정된 가계 지출 데이터 계산
# 각 연도별로 소비자 물가지수를 사용하여 가계 지출 데이터를 물가로 조정
price_adjusted_expenses = Age_specific_per_household_average_household_balance_39_top10_pivot.copy()
for year in price_adjusted_expenses.index:
    price_index = Consumer_Price_Index.loc[year]
    price_adjusted_expenses.loc[year] = price_adjusted_expenses.loc[year] / price_index * 100

# 연도별 증감률 계산
growth_rate = price_adjusted_expenses.pct_change() * 100

# 결과 출력
growth_rate.dropna(inplace=True)  # 첫 번째 행(증감률을 계산할 수 없는 행) 제거
growth_rate

In [ ]:
# 물가 조정된 가계 지출 데이터 계산
price_adjusted_expenses = Age_specific_per_household_average_household_balance_39_top10_pivot.copy()
for year in price_adjusted_expenses.index:
    price_index = Consumer_Price_Index.loc[year]
    price_adjusted_expenses.loc[year] = price_adjusted_expenses.loc[year] / price_index * 100

# 2019년 대비 증감률 계산
base_year = 2019
growth_rate_since_2019 = (price_adjusted_expenses.loc[base_year:] / price_adjusted_expenses.loc[base_year] - 1) * 100

# 결과 출력
growth_rate_since_2019.dropna(inplace=True)  # 필요한 경우 NaN 값 제거
growth_rate_since_2019

1. **인테리어 관련 품목의 변화**:
   - '주거, 수도, 광열': 2020년(5.45% 증가), 2021년(1.74% 증가), 2022년(0.84% 감소)
   - '가정용품 및 가사서비스': 2020년(12.88% 증가), 2021년(8.41% 증가), 2022년(5.56% 감소)
   - '기타 상품 및 서비스': 2020년(6.68% 감소), 2021년(5.04% 감소), 2022년(11.76% 감소)

2. **나머지 품목들의 변화**:
   - '식료품 및 비주류음료': 2020년(10.66% 증가), 2021년(5.04% 증가), 2022년(4.99% 감소)
   - '주류 및 담배': 2020년(6.99% 증가), 2021년(8.66% 증가), 2022년(4.44% 증가)
   - '의류 및 신발': 2020년(10.60% 감소), 2021년(1.25% 감소), 2022년(0.54% 증가)
   - '보건': 2020년(3.70% 증가), 2021년(17.20% 증가), 2022년(11.84% 증가)
   - '교통': 2020년(5.85% 증가), 2021년(7.78% 감소), 2022년(3.49% 감소)
   - '통신': 2020년(3.99% 증가), 2021년(12.51% 증가), 2022년(13.14% 증가)
   - '오락 및 문화': 2020년(10.68% 감소), 2021년(8.50% 감소), 2022년(3.35% 증가)
   - '교육': 2020년(13.76% 감소), 2021년(0.12% 증가), 2022년(15.21% 증가)
   - '음식 및 숙박': 2020년(7.63% 감소), 2021년(4.08% 감소), 2022년(2.62% 증가)

3. **그룹 간 차이의 인사이트**:
   - 인테리어 관련 품목들은 팬데믹 기간 동안 초기에 증가한 후 점차 감소하는 추세를 보였습니다. 이는 집에 머무는 시간이 증가하면서 초기에는 주거 공간에 대한

 투자가 증가했으나, 시간이 지나면서 감소한 것으로 해석됩니다.
   - 나머지 품목들은 각각 다양한 추세를 보였습니다. 식료품 및 비주류음료는 초기에 증가한 후 감소했고, 의류 및 신발은 시간이 지남에 따라 회복세를 보였습니다. 보건과 통신은 지속적으로 증가하는 경향을 보였습니다.

이러한 분석을 통해 인테리어 업체는 현재 시장의 소비 트렌드와 변화하는 고객의 필요를 이해할 수 있으며, 이를 바탕으로 보다 효과적인 마케팅 전략과 제품 개발을 할 수 있을 것입니다.

1. **인테리어 관련 품목의 증감률**:
   - '주거, 수도, 광열': 2020년(5.45% 증가), 2021년(1.74% 증가), 2022년(0.84% 감소)
   - '가정용품 및 가사서비스': 2020년(12.88% 증가), 2021년(8.41% 증가), 2022년(5.56% 감소)
   - '기타 상품 및 서비스': 2020년(6.68% 감소), 2021년(5.04% 감소), 2022년(11.76% 감소)

2. **인테리어 비관련 품목의 증감률**:
   - '식료품 및 비주류음료': 2020년(10.66% 증가), 2021년(5.04% 증가), 2022년(4.99% 감소)
   - '주류 및 담배': 2020년(6.99% 증가), 2021년(8.66% 증가), 2022년(4.44% 증가)
   - '의류 및 신발': 2020년(10.60% 감소), 2021년(1.25% 감소), 2022년(0.54% 증가)

3. **분석 및 인사이트**:
   - **인테리어 관련 품목**:
     - 팬데믹 기간 동안 '가정용품 및 가사서비스' 지출이 크게 증가했으며, 이는 집에서 보내는 시간이 증가함에 따라 가정 내 환경을 개선하려는 수요가 반영된 결과로 해석됩니다.
     - '주거, 수도, 광열' 분야도 증가세를 보였으나, '가정용품 및 가사서비스'보다는 상대적으로 낮은 증가율을 보였습니다. 이는 주거 관련 기본적인 지출이 다소 안정적인 경향을 보임을 나타냅니다.
     - '기타 상품 및 서비스'는 감소 추세를 보였는데, 이는 팬데믹 기간 동안 필수적이지 않은 서비스나 상품에 대한 지출이 줄었음을 의미할 수 있습니다.

   - **인테리어 비관련 품목**:
     - '식료품 및 비주류음료'와 '주류 및 담배'는 팬데믹 기간 동안 증가세를 보였으며, 이는 집에서의 식사 증가와 함께 집에서 보내는 시간이 많아진 것과 관련이 있을 수 있습니다.
     - '의류 및 신발'은 감소한 후 점차 회복세를 보였는데, 이는 팬데믹 초기에 외출 감소로 인한 영향과 점차적인 일상 회복에 따른 소비 회복을 나타냅니다.

이러한 분석을 통해 인테리어 관련 업체는 팬데믹 기간 동안 가정용품과 관련 서비스에 대한 수요 증가를 인지하고, 이에 맞춘 제품 개발 및 마케팅 전략을 수립할 수 있습니다. 반면, 기타 상품 및 서비스에 대한 감소 추세는 시장 전략을 재조정할 필요성을 시사합니다.

In [ ]:
# 인테리어 관련 품목과 비관련 품목의 증감률 데이터 준비
interior_related = growth_rate_since_2019[['04.주거 · 수도 · 광열', '05.가정용품 · 가사서비스', '12.기타상품 · 서비스']]
non_interior_related = growth_rate_since_2019[['01.식료품 · 비주류음료', '02.주류 · 담배', '03.의류 · 신발', '06.보건', '07.교통', '08.통신', '09.오락 · 문화', '10.교육', '11.음식 · 숙박']]

# 시각화
plt.figure(figsize=(14, 8))

# 인테리어 관련 품목 시각화
for column in interior_related.columns:
    plt.plot(interior_related.index, interior_related[column], marker='o', label=column)

# 비관련 품목 시각화
for column in non_interior_related.columns:
    plt.plot(non_interior_related.index, non_interior_related[column], marker='x', linestyle='--', label=column)

plt.title('인테리어 관련 품목 및 비관련 품목의 증감률 비교 (2019년 대비)')
plt.xlabel('연도')
plt.ylabel('증감률 (%)')
plt.legend(loc='upper left')
plt.grid(True)
plt.show()


- 펜데믹 이후 인테리어 관련 품목 소비 감소

---